In [ ]:
# Import dependencies
import pandas as pd
import psycopg2
import boto3
import re

In [ ]:
# Get the S3 keys from config file
from config import aws_access_key, aws_secret_access_key, postgres_key

In [ ]:
# Download census file from S3 bucket
s3 = boto3.client("s3", aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_access_key")
s3.download_file("evanmcgee-projectbucket", "census_2015_raw.csv", "census_2015_raw.csv")

In [ ]:
# Read into a DataFrame
census_df = pd.read_csv("census_2015_raw.csv")

In [ ]:
# Drop the id column
census_df = census_df.drop("id", axis=1)

In [ ]:
# Clean up the Geographic Area Name column values
census_df["Geographic Area Name"] = census_df["Geographic Area Name"].str.replace("Congressional District","")
census_df["Geographic Area Name"] = census_df["Geographic Area Name"].str.replace("\(114th Congress\),", "")
census_df["Geographic Area Name"] = census_df["Geographic Area Name"].str.replace("\(at Large\)", "")

In [ ]:
# Remove columns starting with Margin of Error
census_df = census_df.loc[:,~census_df.columns.str.startswith("Margin of Error")]

In [ ]:
# Remove columns starting with Percent Margin of Error
census_df = census_df.loc[:,~census_df.columns.str.startswith("Percent Margin of Error")]

In [ ]:
# Extract state names using regex and add to new column
census_df["state"] = census_df["Geographic Area Name"].str.extract(r"([A-Z][a-z]+(?: +[A-Z][a-z]+)*)")

In [ ]:
# Extract district numbers using regex and add to new column
census_df["district"] = census_df["Geographic Area Name"].str.extract(r"(\d+)")

In [ ]:
# Drop the Geographic Area Name column
census_df = census_df.drop(columns="Geographic Area Name")

In [ ]:
# Strip the spaces from the state column
census_df["state"] = census_df["state"].str.replace(" ","")

In [ ]:
# Remove D.C. and Puerto Rico from the DataFrame
census_df = census_df[census_df["state"] != "DelegateDistrictDistrict"]
census_df = census_df[census_df["state"] != "ResidentCommissionerDistrictPuertoRico"]

In [ ]:
# Convert state column values to abbreviations

state_list = census_df["state"]

new_states = []

for state in state_list:
    if state == "Alabama":
        new_states.append("AL")
    if state == "Alaska":
        new_states.append("AK")
    if state == "Arizona":
        new_states.append("AZ")
    if state == "Arkansas":
        new_states.append("AR")
    if state == "California":
        new_states.append("CA")
    if state == "Colorado":
        new_states.append("CO")
    if state == "Connecticut":
        new_states.append("CT")
    if state == "Delaware":
        new_states.append("DE")
    if state == "Florida":
        new_states.append("FL")
    if state == "Georgia":
        new_states.append("GA")
    if state == "Hawaii":
        new_states.append("HI")
    if state == "Idaho":
        new_states.append("ID")
    if state == "Illinois":
        new_states.append("IL")
    if state == "Indiana":
        new_states.append("IN")
    if state == "Iowa":
        new_states.append("IA")
    if state == "Kansas":
        new_states.append("KS")
    if state == "Kentucky":
        new_states.append("KY")
    if state == "Louisiana":
        new_states.append("LA")
    if state == "Maine":
        new_states.append("ME")
    if state == "Maryland":
        new_states.append("MD")
    if state == "Massachusetts":
        new_states.append("MA")
    if state == "Michigan":
        new_states.append("MI")
    if state == "Minnesota":
        new_states.append("MN")
    if state == "Mississippi":
        new_states.append("MS")
    if state == "Missouri":
        new_states.append("MO")
    if state == "Montana":
        new_states.append("MT")
    if state == "Nebraska":
        new_states.append("NE")
    if state == "Nevada":
        new_states.append("NV")
    if state == "NewHampshire":
        new_states.append("NH")
    if state == "NewJersey":
        new_states.append("NJ")
    if state == "NewMexico":
        new_states.append("NM")
    if state == "NewYork":
        new_states.append("NY")
    if state == "NorthCarolina":
        new_states.append("NC")
    if state == "NorthDakota":
        new_states.append("ND")
    if state == "Ohio":
        new_states.append("OH")
    if state == "Oklahoma":
        new_states.append("OK")
    if state == "Oregon":
        new_states.append("OR")
    if state == "Pennsylvania":
        new_states.append("PA")
    if state == "RhodeIsland":
        new_states.append("RI")
    if state == "SouthCarolina":
        new_states.append("SC")
    if state == "SouthDakota":
        new_states.append("SD")
    if state == "Tennessee":
        new_states.append("TN")
    if state == "Texas":
        new_states.append("TX")
    if state == "Utah":
        new_states.append("UT")
    if state == "Vermont":
        new_states.append("VT")
    if state == "Virginia":
        new_states.append("VA")
    if state == "Washington":
        new_states.append("WA")
    if state == "WestVirginia":
        new_states.append("WV")
    if state == "Wisconsin":
        new_states.append("WI")
    if state == "Wyoming":
        new_states.append("WY")

In [ ]:
# Add the abbreviations to a new column
census_df["new_state"] = new_states

# Drop the original column
census_df = census_df.drop(columns="state")

In [ ]:
# Replace NANs in district column with 1s
census_df["district"] = census_df["district"].fillna("1")

In [ ]:
# Add a zero to the beginning of single digit numbers in district column
census_df["district"] = census_df["district"].str.zfill(2)

In [ ]:
# Concatenate the state and district columns
census_df["state_district"] = census_df["new_state"] + census_df["district"]

In [ ]:
# Drop the district and new_state columns
census_df = census_df.drop(columns="district")
census_df = census_df.drop(columns="new_state")

In [ ]:
# Move the state_district column to the front of the DataFrame
col_name="state_district"
first_col = census_df.pop(col_name)
census_df.insert(0, col_name, first_col)

In [ ]:
# Create a new dataframe with selected columns

df = census_df[[
    "state_district",
    "Estimate!!SEX AND AGE!!Total population",
    "Percent!!SEX AND AGE!!Total population!!Male",
    "Percent!!SEX AND AGE!!Total population!!Female",
    "Percent!!SEX AND AGE!!Under 5 years",
    "Percent!!SEX AND AGE!!5 to 9 years",
    "Percent!!SEX AND AGE!!10 to 14 years",
    "Percent!!SEX AND AGE!!15 to 19 years",
    "Percent!!SEX AND AGE!!20 to 24 years",
    "Percent!!SEX AND AGE!!25 to 34 years",
    "Percent!!SEX AND AGE!!35 to 44 years",
    "Percent!!SEX AND AGE!!45 to 54 years",
    "Percent!!SEX AND AGE!!55 to 59 years",
    "Percent!!SEX AND AGE!!60 to 64 years",
    "Percent!!SEX AND AGE!!65 to 74 years",
    "Percent!!SEX AND AGE!!75 to 84 years",
    "Percent!!SEX AND AGE!!85 years and over",
    "Estimate!!SEX AND AGE!!Median age (years)",
    "Percent!!SEX AND AGE!!18 years and over",
    "Percent!!SEX AND AGE!!21 years and over",
    "Percent!!SEX AND AGE!!62 years and over",
    "Percent!!SEX AND AGE!!65 years and over",
    "Percent!!SEX AND AGE!!18 years and over!!Male",
    "Percent!!SEX AND AGE!!18 years and over!!Female",
    "Percent!!SEX AND AGE!!65 years and over!!Male",
    "Percent!!SEX AND AGE!!65 years and over!!Female",
    "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone",
    "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Black or African American alone",
    "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone",
    "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Asian alone",
    "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone",
    "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone",
    "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)",
    "Estimate!!Total housing units"
]]

In [ ]:
# Rename the column names

df.columns = [
    "district",
    "total_pop",
    "male_pop_percent",
    "female_pop_percent",
    "under_5_percent",
    "5_to_9_percent",
    "10_to_14_percent",
    "15_to_19_percent",
    "20_to_24_percent",
    "25_to_34_percent",
    "35_to_44_percent",
    "45_to_54_percent",
    "55_to_59_percent",
    "60_to_64_percent",
    "65_to_74_percent",
    "75_to_84_percent",
    "over_85_percent",
    "median_age",
    "18_and_over_percent",
    "21_and_over_percent",
    "62_and_over_percent",
    "65_and_over_percent",
    "18_and_over_male_percent",
    "18_and_over_female_percent",
    "65_and_over_male_percent",
    "65_and_over_female_percent",
    "white_pop_percent",
    "black_pop_percent",
    "american_indian_alaska_native_pop_percent",
    "asian_pop_percent",
    "native_hawaiian_pacific_islander_pop_percent",
    "other_race_pop_percent",
    "hispanic_pop_percent",
    "total_housing_units"
]

In [ ]:
# Add an election year column
data = 2016

df.insert(1, "election_year", data)

In [ ]:
# Export to csv
df.to_csv("Data/census_2015.csv", index=False)

In [ ]:
# Upload df to sql database
conn = psycopg2.connect("host=projectdatabase.c1ibogbs35nd.us-east-2.rds.amazonaws.com dbname=election_results user=postgres password=postgres_key")
cur = conn.cursor()
with open("Data/census_2015.csv", "r") as f:
    next(f) # Skip the header row.
    cur.copy_from(f, "census_2015", sep=",")

conn.commit()